## Problem Statement

*Exercise 2.12 from Operations Research: Models and Methods by Jensen & Bard*

Ten jobs are to be completed by three workers during the next week. Each worker has a 40-hour work week. The times for the workers to complete the jobs are shown in the table. The values in the cells assume that each job is completed by a single worker; however, jobs can be shared, with completion times being determined proportionally If no entry exists in a particular cell, it means that the corresponding job cannot be performed by the corresponding worker. Set up and solve an LP model that will determine the optimal assignment of workers to jobs. The goal is to minimize the total time required to complete all the jobs.

| Workers \ Tasks |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  8 |  9 | 10 |
|:---------------:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|
| A               |  - |  7 |  3 |  - |  - | 18 | 13 |  6 |  - |  9 |
| B               | 12 |  5 |  - | 12 |  4 | 22 |  - | 17 | 13 |  - |
| C               | 18 |  - |  6 |  8 | 10 |  - | 19 |  - |  8 | 15 |

## Import

In [ ]:
from collections import defaultdict

import gurobipy as grb
import pandas as pd

## Define Data

In [ ]:
workers = {'A', 'B', 'C'}

tasks = set(range(1, 11))

c = defaultdict(lambda: 1000, {
    ('A',  2):  7,
    ('A',  3):  3,
    ('A',  6): 18,
    ('A',  7): 13,
    ('A',  8):  6,
    ('A', 10):  9,
    ('B',  1): 12,
    ('B',  2):  5,
    ('B',  4): 12,
    ('B',  5):  4,
    ('B',  6): 22,
    ('B',  8): 17,
    ('B',  9): 13,
    ('C',  1): 18,
    ('C',  3):  6,
    ('C',  4):  8,
    ('C',  5): 10,
    ('C',  7): 19,
    ('C',  9):  8,
    ('C', 10): 15,
})

max_hours = 40

## Model
Define $W$ as the set of workers and $T$ as the sets of tasks. Also, define $c_{wt}$ as the number of hours worker $w$ requires to complete task $t$. (Note that we do not explicitly prohibit a worker from completiting as task; rather, we make the cost arbitrarily large if worker $w$ is unable to perform task $t$.) Let $x_{wt}$ be the proportion of task $t$ that is completed by worker $j$. Let $H$ be the max number of hours that any single worker may log in a week. We formulate as follows.


$$
\begin{alignat*}{3}
\text{minimize  }  & \sum_{w \in W} \sum_{t \in T} c_{wt} x_{wt} && \\
\text{subject to  }
& \sum_{t \in T} c_{wt} x_{wt} \le H,
&& \qquad \forall w \in W \\
& \sum_{w \in W} x_{wt} = 1
&& \qquad \forall t \in T \\
& 0 \le x_{wt} \le 1,
&& \qquad \forall w \in W, \forall t \in T
\end{alignat*}
$$

## Implement

In [ ]:
model = grb.Model()

In [ ]:
x = model.addVars(workers, tasks, vtype='C', lb=0, ub=1)
model.update()

In [ ]:
expr = sum(c[w, t] * x[w, t]
           for w in workers for t in tasks)
model.setObjective(sense=grb.GRB.MINIMIZE, expr=expr)

In [ ]:
for t in tasks:
    lhs = sum(x[w, t] for w in workers)
    rhs = 1
    model.addConstr(lhs == rhs)

In [ ]:
for w in workers:
    lhs = sum(c[w, t] * x[w, t] for t in tasks)
    rhs = max_hours
    model.addConstr(lhs <= rhs)

In [ ]:
model.update()

## Solve and Postprocess

In [ ]:
model.optimize()

In [ ]:
df = pd.DataFrame(index=pd.MultiIndex.from_tuples(x, names=['w', 't']))
df['x'] = [x[k].x for k in df.index]
df['c'] = [c[k] for k in df.index]

In [ ]:
(df['c'] * df['x']).unstack('t')

In [ ]:
(df['c'] * df['x']).groupby('w').sum().to_frame()

In [ ]:
df['x'].groupby('t').sum().to_frame().T